In [1]:
import pyspark

In [2]:
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder \
                    .master("local[1]") \
                    .appName("") \
                    .getOrCreate()

23/02/21 18:06:00 WARN Utils: Your hostname, JosePC resolves to a loopback address: 127.0.1.1; using 192.168.15.30 instead (on interface wlo1)
23/02/21 18:06:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/21 18:06:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/21 18:06:02 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
import datetime

In [5]:
from pyspark.sql import Row

In [6]:
users = [
    {
        "id": 1,
        "first_name": "Corrie",
        "last_name": "Van den Oord",
        "email": "cvandenoord@etsy.com",
        "phone_numbers": Row(mobile="+1 234 567 8901", home="+1 234 567 8911"),
        "courses": [1, 2],
        "is_costumer": True,
        "amount_paid": 1000.55,
        "customer_from": datetime.date(2021, 1, 15),
        "last_updated_ts": datetime.datetime(2021, 2, 10, 1, 15, 0)
    },
    {
        "id": 2,
        "first_name": "Nikolaus",
        "last_name": "Brewitt",
        "email": "nbrewittl@dailyail.co.uk",
        "phone_numbers": Row(mobile="+1 234 567 8923", home="+1 234 567 8934"),
        "courses": [3],
        "is_costumer": True,
        "amount_paid": 900.00,
        "customer_from": datetime.date(2021, 2, 14),
        "last_updated_ts": datetime.datetime(2021, 2, 18, 3, 33, 0)
    },
    {
        "id": 3,
        "first_name": "Orelie",
        "last_name": "Penney", 
        "email": "openney2@vistaprint.com",
        "phone_numbers": Row(mobile="+1 714 512 9752", home="+1 714 512 6601"),
        "courses": [2, 3],
        "is_costumer": True,
        "amount_paid": 850.55,
        "customer_from": datetime.date(2021, 1, 21),
        "last_updated_ts": datetime.datetime(2021, 3, 15, 15, 16, 55)
    },
    {
        "id": 4,
        "first_name": "Ashby",
        "last_name": "Maddocks",
        "email": "amaddocks3@home.pl",
        "phone_numbers": Row(mobile=None, home=None),
        "courses": [],
        "is_costumer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 10, 17, 45, 30)
    },
    {
        "id": 5,
        "first_name": "Kurt",
        "last_name": "Rome",
        "email": "krome4@shutterfly.com",
        "phone_numbers": Row(mobile= "+1 817 934 7142", home=None),
        "courses": [],
        "is_costumer": False,
        "amount_paid": None,
        "customer_from": None,
        "last_updated_ts": datetime.datetime(2021, 4, 2, 0, 55, 18)
    }
]

In [7]:
import pandas as pd

In [8]:
spark.conf.set('spark.sql.execution.arrowpyspark.enabled', False)

In [9]:
users_df = spark.createDataFrame(pd.DataFrame(users))

In [10]:
users_df.show()

+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
| id|first_name|   last_name|               email|       phone_numbers|courses|is_costumer|amount_paid|customer_from|    last_updated_ts|
+---+----------+------------+--------------------+--------------------+-------+-----------+-----------+-------------+-------------------+
|  1|    Corrie|Van den Oord|cvandenoord@etsy.com|{+1 234 567 8901,...| [1, 2]|       true|    1000.55|   2021-01-15|2021-02-10 01:15:00|
|  2|  Nikolaus|     Brewitt|nbrewittl@dailyai...|{+1 234 567 8923,...|    [3]|       true|      900.0|   2021-02-14|2021-02-18 03:33:00|
|  3|    Orelie|      Penney|openney2@vistapri...|{+1 714 512 9752,...| [2, 3]|       true|     850.55|   2021-01-21|2021-03-15 15:16:55|
|  4|     Ashby|    Maddocks|  amaddocks3@home.pl|        {null, null}|     []|      false|        NaN|         null|2021-04-10 17:45:30|
|  5|      Kurt|        Rome|krome

In [11]:
from pyspark.sql.functions import col, lit, concat

In [12]:
full_name_col = concat(col('first_name'), lit(', '), col('last_name'))

In [13]:
full_name_col

Column<'concat(first_name, , , last_name)'>

In [16]:
full_name_alias = full_name_col.alias('full_name')

In [17]:
users_df.select('id', full_name_alias).show()

+---+--------------------+
| id|           full_name|
+---+--------------------+
|  1|Corrie, Van den Oord|
|  2|   Nikolaus, Brewitt|
|  3|      Orelie, Penney|
|  4|     Ashby, Maddocks|
|  5|          Kurt, Rome|
+---+--------------------+



In [18]:
users_df.select('id', 'customer_from').show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|   2021-01-15|
|  2|   2021-02-14|
|  3|   2021-01-21|
|  4|         null|
|  5|         null|
+---+-------------+



In [19]:
from pyspark.sql.functions import date_format

In [20]:
date_format('customer_from', 'yyyyMMdd')

Column<'date_format(customer_from, yyyyMMdd)'>

In [21]:
date_format('customer_from', 'yyyyMMdd').cast('int')

Column<'CAST(date_format(customer_from, yyyyMMdd) AS INT)'>

In [22]:
date_format('customer_from', 'yyyyMMdd').cast('int').alias('customer_from')

Column<'CAST(date_format(customer_from, yyyyMMdd) AS INT) AS customer_from'>

In [23]:
customer_from_alias = date_format('customer_from', 'yyyyMMdd').cast('int').alias('customer_from')

In [24]:
users_df.select('id', customer_from_alias).show()

+---+-------------+
| id|customer_from|
+---+-------------+
|  1|     20210115|
|  2|     20210214|
|  3|     20210121|
|  4|         null|
|  5|         null|
+---+-------------+



In [25]:
users_df.select('id', customer_from_alias).dtypes

[('id', 'bigint'), ('customer_from', 'int')]